# Mixing TensorFlow models with GPflow

This notebook explores two ways to combine TensorFlow neural networks with GPflow models.


In [61]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import gpflow
from gpflow.ci_utils import ci_niter
from scipy.cluster.vq import kmeans2

iterations = ci_niter(100)

## Convolutional network inside a GPflow model
Here we'll use the GPflow functionality, but put a non-GPflow model inside the kernel.

In [83]:
from typing import Dict, Optional, Tuple
import tensorflow as tf
import tensorflow_datasets as tfds
import gpflow
from gpflow.config import default_float

def map_fn(input_slice: Dict[str, tf.Tensor]):
    updated = input_slice
    image = tf.cast(updated["image"], tf.float32) / 255.
    label = tf.cast(updated["label"], default_float())
    return image, label

autotune = tf.data.experimental.AUTOTUNE

original_dataset, info = tfds.load(name="mnist", split=tfds.Split.TRAIN, with_info=True)
dataset = original_dataset\
    .shuffle(1024)\
    .batch(32, drop_remainder=True)\
    .map(map_fn, num_parallel_calls=autotune)\
    .prefetch(autotune)

total_num_data = info.splits["train"].num_examples

A vanilla ConvNet. This gets 97.3% accuracy on MNIST when used on its own (+ final linear layer) after 20K iterations

In [84]:
class KernelWithConvNN(gpflow.kernels.Kernel):
    def __init__(self, input_shape: Tuple, output_dim: int, base_kernel: gpflow.kernels.Kernel):
        super().__init__()
        with self.name_scope:
            self.base_kernel = base_kernel
            self.cnn = tf.keras.Sequential([
                tf.keras.layers.Conv2D(filters=32, kernel_size=(28, 28), padding="same", activation="relu"),
                tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2),
                tf.keras.layers.Conv2D(filters=64, kernel_size=(5, 5), padding="same", activation="relu"),
                tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(output_dim, activation="relu"),
                tf.keras.layers.Lambda(lambda x: tf.cast(x, default_float()))
            ])
            self.cnn.build(input_shape)
    
    def K(self, a_input: tf.Tensor, b_input: Optional[tf.Tensor] = None, presliced: bool = False) -> tf.Tensor:
        transformed_a = self.cnn(a_input)
        transformed_b = self.cnn(b_input) if b_input is not None else b_input
        return self.base_kernel.K(transformed_a, transformed_b, presliced)
    
    def K_diag(self, a_input: tf.Tensor, presliced: bool = False) -> tf.Tensor:
        return self.base_kernel.K_diag(a_input, presliced)

$K_{uf}$ is in ConvNN output space, therefore we need to update `Kuf` multi dispatch

In [85]:
class KernelSpaceInducingPoints(gpflow.inducing_variables.InducingPoints):
    pass

@gpflow.covariances.Kuf.register(KernelSpaceInducingPoints, KernelWithConvNN, object)
def Kuf(inducing_variable, kernel, a_input):
    return kernel.base_kernel(inducing_variable, kernel.cnn(Xnew))

In [97]:
num_mnist_classes = 10
output_dim = 5
num_inducing_points = 100
input_shape = (None, 28, 28, 1)
dataset_subset = next(iter(dataset.batch(32)))
input_images = tf.reshape(dataset_subset[0], [-1, *input_shape[1:]])

kernel = KernelWithConvNN(input_shape, output_dim, gpflow.kernels.SquaredExponential())
likelihood = gpflow.likelihoods.MultiClass(num_mnist_classes)
inducing_variable = kmeans2(input_images.numpy().reshape(-1, 28 * 28),
                            num_inducing_points, minit='points')[0]

model = gpflow.models.SVGP(
    kernel, likelihood,
    inducing_variable=inducing_variable,
    num_data=total_num_data,
    num_latent=num_mnist_classes)

data_iterator = iter(dataset)
adam_opt = tf.optimizers.Adam(0.01)

@tf.function(autograph=False)
def loss_cb():
    x, y = next(data_iterator)
    return -model.elbo(x, y)


@tf.function(autograph=False)
def optimization_step():
    adam_opt.minimize(loss_cb, var_list=model.trainable_variables)

In [94]:
d = next(iter(dataset))

In [98]:
x, y = next(data_iterator)
model.elbo(x, y)

ValueError: Incompatible type conversion requested to type 'float64' for variable of type 'float32'

In [65]:
# Predictions
m, v = model.predict_y(Mnist.Xtest)
preds = np.argmax(m, 1).reshape(Mnist.Ytest.shape)
correct = (preds == Mnist.Ytest.astype(int))
acc = np.average(correct.astype(float)) * 100.

print('Accuracy is {:.4f}%'.format(acc))

NameError: name 'model' is not defined

In [6]:
def ex2b():
    fX_dim = 5  
    minibatch_size = notebook_niter(1000, test_n=10)
    M = notebook_niter(100, test_n=5)

    # Unfortunately, only float32 and lower are supported by the Conv layers 
    f = lambda x: tf.cast(cnn_fn(tf.cast(x, tf.float32), fX_dim), float_type)
    kern = KernelWithNN(gpflow.kernels.Matern32(fX_dim), f)
    
    # Reset inducing (these live in a different space to X, so we need to be careful here)
    ind = np.random.choice(Mnist.X.shape[0], minibatch_size, replace=False)
    
    # Currently a hack is needed due to model initialisation
    feat = KernelSpaceInducingPoints(np.empty((M, fX_dim)))
    #feat = FFeature(Z_0)  # Ideally, we could move the calculation of Z_0
    
    # Build the model 

    lik = gpflow.likelihoods.MultiClass(Mnist.Nclasses)

    #Z = kmeans2(Mnist.X, M, minit='points')[0]

    model = NN_SVGP(Mnist.X, Mnist.Y, kern, lik, feat=feat, num_latent=Mnist.Nclasses, minibatch_size=minibatch_size)

    fZ = model.kern.compute_f(Mnist.X[ind])
    # Z_0 = kmeans2(fZ, M)[0] might fail
    Z_0 = fZ[np.random.choice(len(fZ), M, replace=False)]
    model.feature.Z = Z_0

    # Use GPflow wrappers to train. Note all session handling is done for us
    gpflow.training.AdamOptimizer(0.001)

    # Predictions
    m, v = model.predict_y(Mnist.Xtest)
    preds = np.argmax(m, 1).reshape(Mnist.Ytest.shape)
    correct = preds == Mnist.Ytest.astype(int)
    acc = np.average(correct.astype(float)) * 100.

    print('Accuracy is {:.4f}%'.format(acc))

gpflow.reset_default_graph_and_session()
ex2b()

Accuracy is 97.5600%


## Example 2: A GPflow model on top of a TensorFlow model
Now we'll do things the other way; take a model implemented in TensorFlow and explain how to put a GPflow model on top.

In [7]:
def ex2():
    minibatch_size = notebook_niter(1000, test_n=10)
    gp_dim = 5
    M = notebook_niter(100, test_n=5)

    # Placeholders
    X = tf.placeholder(tf.float32, [minibatch_size, Mnist.input_dim])  # Fixed shape so num_data works in SVGP
    Y = tf.placeholder(tf.float32, [minibatch_size, 1])
    Xtest = tf.placeholder(tf.float32, [None, Mnist.input_dim])

    # Build graph

    with tf.variable_scope('cnn'):
        f_X = tf.cast(cnn_fn(X, gp_dim), dtype=float_type)

    with tf.variable_scope('cnn', reuse=True):
        f_Xtest = tf.cast(cnn_fn(Xtest, gp_dim), dtype=float_type)

    gp_model = gpflow.models.SVGP(f_X, tf.cast(Y, dtype=float_type), 
                                  gpflow.kernels.RBF(gp_dim), gpflow.likelihoods.MultiClass(Mnist.Nclasses), 
                                  Z=np.zeros((M, gp_dim)), # we'll set this later
                                  num_latent=Mnist.Nclasses)

    loss = -gp_model.likelihood_tensor

    m, v = gp_model._build_predict(f_Xtest)
    my, yv = gp_model.likelihood.predict_mean_and_var(m, v)

    with tf.variable_scope('adam'):
        opt_step = tf.train.AdamOptimizer(0.001).minimize(loss)

    tf_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='adam')
    tf_vars += tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='cnn')

    # Initialise
    sess = tf.Session()
    sess.run(tf.variables_initializer(var_list=tf_vars))
    gp_model.initialize(session=sess)
    
    # Reset inducing (these live in a different space to X, so we need to be careful here)
    ind = np.random.choice(Mnist.X.shape[0], minibatch_size, replace=False)

    fZ = sess.run(f_X, feed_dict={X:Mnist.X[ind]})
    # Z_0 = kmeans2(fZ, M)[0] might fail
    Z_0 = fZ[np.random.choice(len(fZ), M, replace=False)]

    def set_gp_param(param, value):
        sess.run(tf.assign(param.unconstrained_tensor, param.transform.backward(value)))

    set_gp_param(gp_model.feature.Z, Z_0)

    # Train
    for i in range(ITERATIONS):
        ind = np.random.choice(Mnist.X.shape[0], minibatch_size, replace=False)
        sess.run(opt_step, feed_dict={X:Mnist.X[ind], Y:Mnist.Y[ind]})

    # Predict
    preds = np.argmax(sess.run(my, feed_dict={Xtest:Mnist.Xtest}), 1).reshape(Mnist.Ytest.shape)
    correct = preds == Mnist.Ytest.astype(int)
    acc = np.average(correct.astype(float)) * 100.
    print('acc is {:.4f}'.format(acc))

gpflow.reset_default_graph_and_session()
ex2()

gpflow.reset_default_graph_and_session()

acc is 98.5900
